# Reservoir Operations under Uncertainty

## Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats as ss
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline

## Define simulation parameters

In [ ]:
# capacity of reservoir
K = 200

# maximum release
Rmax = 150

# vector of storage values; start with full reservoir
S = np.empty(7)
S[0] = K

# vector of release decisions to be populated by user; initialize with empty values
R = np.empty([6])

# initialize variables to store flow forecasts
Zfcst = np.empty([6]) # standard normal random variable
Qfcst = np.empty([6]) # real-space median flow forecast
Qfcst025 = np.empty([6]) # 2.5%-ile flow forecast
Qfcst975 = np.empty([6]) # 97.5%-ile flow forecast

# initialize variables to store simulated flows
Zsim = np.empty([6])
Qsim = np.empty([6])

# standard normal random variables corresponding to 2.5%-ile and 97.5%-ile
zCrit025 = ss.norm.ppf(0.025)
zCrit975 = ss.norm.ppf(0.975)

# parameters of random inflow
rho = 0.5 # lag-1 autocorrelation coefficient
logAvg = np.array([2.9, 3.9, 4.6, 4.6, 4.1, 3.7]) # average of log-space flows for first 6 time steps
logSd = np.array([0.5, 0.4, 0.3, 0.4, 0.3, 0.2]) # standard deviation of log-space flows for first 6 time steps

## Define function to plot forecasts and reservoir storage throughout the simulation

In [ ]:
# function to plot forecast and storage trajectory
def plotForecast(t, S, K, Qfcst, Qfcst025, Qfcst975, Qsim):
    fig = plt.figure()
    ax = fig.add_subplot(1,2,1) # forecast plot
    if t != 0:
        ax.scatter(range(1,t+1),Qsim[0:t],color="#e6550d") # observed inflows (points)
        ax.plot(range(1,t+1),Qsim[0:t],color="#e6550d") # observed inflows (line)
        ax.plot(range(t,7),Qfcst[(t-1)::],color="#3182bd") # median inflow forecast
        ax.fill_between(range(t,7),Qfcst025[(t-1)::],Qfcst975[(t-1)::],
                        color="#9ecae1") # 95% CI on inflow forecast
        ax.set_title("Observed and Forecast\nInflow at Month " + str(t))
    else:
        ax.plot(range(1,7),Qfcst,color="#3182bd") # median inflow forecast
        ax.fill_between(range(1,7),Qfcst025,Qfcst975,color="#9ecae1") # 95% CI on inflow forecast
        ax.set_title("Forecast Inflow at Month " + str(t))
    ax.set_ylim([0,250])
    ax.set_ylabel("Forecasted inflow")
    ax.set_xlabel("Month")
    ax.set_xticks(range(7))
    
    ax = fig.add_subplot(1,2,2) # storage plot
    ax.scatter(range(t+1),S[0:(t+1)],color="#31a354") # observed storage (points)
    ax.plot(range(t+1),S[0:(t+1)],color="#31a354") # observed storage (line)
    ax.plot(range(7),np.repeat(K,7),color="k") # reservoir storage capacity
    ax.set_ylim([0,250])
    ax.set_ylabel("Storage")
    ax.set_xlabel("Month")
    ax.set_title("Storage up to Month " + str(t))
    ax.set_xticks(range(7))
    
    fig.tight_layout()
    
    plt.show()

## Run 6-month simulation, starting with a full reservoir. The goal is to have the highest reservoir level at the end of the simulation without overtopping the dam.

In [ ]:
# set random seed
seed = int(input("Enter a random seed: "))
np.random.default_rng(seed)

# loop through 6 months
for t in range(6):
    print("\nMonth=" + str(t))
    print("Current Storage: " + str(S[t]))
    print("Reservoir capacity: " + str(K))
    # replace forecast up to time t with observations
    if t > 0:
        Zfcst[t-1] = Zsim[t-1]
        Qfcst[t-1] = Qsim[t-1]
        Qfcst025[t-1] = Qsim[t-1]
        Qfcst975[t-1] = Qsim[t-1]
    
    # generate flow forecast for next 6-t time steps        
    index = []
    for j in range(t,6):
        if j == 0:
            Zfcst[j] = ss.norm.ppf(0.5) # median for first time step
        else:            
            Zfcst[j] = rho*(Zfcst[j-1]) # correlated with past forecast/observation        
        
        Qfcst[j] = int(np.exp(Zfcst[j]*logSd[j] + logAvg[j]))
        Qfcst025[j] = int(np.exp((Zfcst[j]+zCrit025)*logSd[j] + logAvg[j]))
        Qfcst975[j] = int(np.exp((Zfcst[j]+zCrit975)*logSd[j] + logAvg[j]))
        index.append("Month " + str(j+1))        

    # print and plot forecast for user
    df = pd.DataFrame({"Average Fcst": Qfcst[t::], "2.5%-ile": Qfcst025[t::], 
               "97.5%-ile": Qfcst975[t::]}, index = index)
    print(df)
    plotForecast(t, S, K, Qfcst, Qfcst025, Qfcst975, Qsim)
                    
    # get release decision from user
    while True:
        R[t] = input("How much of the current storage + next month's inflow \n"
        f"would you like to release over the next month? (max release = {Rmax})\n")
        if R[t] > Rmax:
            print(f"Release must be < {Rmax}\n")
        elif R[t] < 0:
            print("Release must be >= 0\n")
        else:
            break
        
    # generate true inflow
    if t == 0:
        Zsim[t] = ss.norm.rvs(0,1,1)
    else:
        Zsim[t] = rho*(Zsim[t-1]) + ss.norm.rvs(0,1,1)*np.sqrt(1-rho**2)
        
    Qsim[t] = int(np.exp(Zsim[t]*logSd[t] + logAvg[t]))
    
    print("Inflow: " + str(Qsim[t]))
    print("Release: " + str(min(R[t], S[t] + Qsim[t])))
    
    # simulate next storage value
    S[t+1] = S[t] + Qsim[t] - min(R[t], S[t] + Qsim[t])
    
    if S[t+1] > K:
        print("Oh no! You overtopped the dam")
        break

print("Final reservoir storage: " + str(S[t+1]))
print("Reservoir capacity: " + str(K))

# update observations
Zfcst[t] = Zsim[t]
Qfcst[t] = Qsim[t]
Qfcst025[t] = Qsim[t]
Qfcst975[t] = Qsim[t]
plotForecast(t+1, S, K, Qfcst, Qfcst025, Qfcst975, Qsim)